In [2]:
import pandas as pd

In [3]:
sez = pd.read_csv("joined_sez_map.csv")

In [30]:
pop = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/population/p_landscan_pop.csv")

In [31]:
pop.rename(columns = {"asdf_id" : "id"}, inplace=True)

In [32]:
column_filter = pop[pop.columns[pop.columns.str.contains("land")]]

In [33]:
for col in column_filter:
    new_name = "pop" + col.split(".", 1)[1]
    pop.rename(columns = {col : new_name}, inplace = True)

In [34]:
pop_sum = pop[pop.columns[pop.columns.str.contains("sum|id")]]

In [35]:
pop_sum.columns = pop_sum.columns.str.replace(".sum$", "", regex=True)

In [36]:
pop_sum.dtypes

id           int64
pop2000    float64
pop2001    float64
pop2002    float64
pop2003    float64
pop2004    float64
pop2005    float64
pop2006    float64
pop2007    float64
pop2008    float64
pop2009    float64
pop2010    float64
pop2011    float64
pop2012    float64
pop2013    float64
pop2014    float64
pop2015    float64
pop2016    float64
pop2017    float64
pop2018    float64
pop2019    float64
pop2020    float64
pop2021    float64
dtype: object

In [16]:
#prep = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm2/precipitation/precipitation_00_22.csv")

In [17]:
#prep_filter = prep[prep.columns[prep.columns.str.contains("pre_yearly")]]

In [18]:
#for col in prep_filter: 
#    new_name = "prep" + col.split(".", 1)[1]
#    prep.rename(columns = {col : new_name}, inplace = True)

In [19]:
#prep.columns = prep.columns.str.replace(".mean$", "", regex=True)

In [20]:
#prep.rename(columns = {"asdf_id" : "id"}, inplace=True)

In [21]:
#prep = prep[prep.columns[prep.columns.str.contains("prep|id")]]

In [84]:
temp = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/temperature/p_landsurface_temp.csv")

In [85]:
temp.rename(columns = {"asdf_id" : "id"}, inplace=True)

In [88]:
temp_filter = temp[temp.columns[temp.columns.str.contains("mod11c3_061")]]

In [90]:
temp_filter = temp_filter[temp_filter.columns[temp_filter.columns.str.contains(".mean.1")]]

In [92]:
for col in temp_filter: 
    new_name = "temp" + col.split(".", 1)[1]
    temp.rename(columns = {col : new_name}, inplace=True)

In [81]:
temp.columns = temp.columns.str.replace(".mean.1", "", regex=True)

In [82]:
temp = temp[temp.columns[temp.columns.str.contains("temp|id")]]

In [93]:
temp.columns

Index(['id', 'modis_lst_mod11c3_061_day_annual_mean.2000.max',
       'modis_lst_mod11c3_061_day_annual_mean.2000.mean',
       'modis_lst_mod11c3_061_day_annual_mean.2000.min',
       'modis_lst_mod11c3_061_day_annual_mean.2001.max',
       'modis_lst_mod11c3_061_day_annual_mean.2001.mean',
       'modis_lst_mod11c3_061_day_annual_mean.2001.min',
       'modis_lst_mod11c3_061_day_annual_mean.2002.max',
       'modis_lst_mod11c3_061_day_annual_mean.2002.mean',
       'modis_lst_mod11c3_061_day_annual_mean.2002.min',
       ...
       'temp2021.mean.1', 'temp2022.mean.1', 'temp2023.mean.1', 'gqid', 'id',
       'shapeGroup', 'shapeID', 'shapeISO', 'shapeName', 'shapeType'],
      dtype='object', length=104)

In [75]:
temp

,id,id,id,id,id,id,id,id,temp2000,temp2001,...,temp2023,gqid,id,id,id,id,id,id,id,id
0,0,0,0,0,0,0,0,0,300.565424,300.506478,...,300.341336,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,301.359654,300.930243,...,301.276388,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2,302.439574,302.049416,...,301.931481,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,301.368647,300.343604,...,301.151915,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,298.775821,299.081308,...,298.539112,4,4,4,4,4,4,4,4,4
5,5,5,5,5,5,5,5,5,301.658911,301.352985,...,301.910803,5,5,5,5,5,5,5,5,5
6,6,6,6,6,6,6,6,6,299.467922,298.950390,...,299.717685,6,6,6,6,6,6,6,6,6
7,7,7,7,7,7,7,7,7,305.302674,304.399808,...,304.305546,7,7,7,7,7,7,7,7,7
8,8,8,8,8,8,8,8,8,302.831851,302.694366,...,301.182097,8,8,8,8,8,8,8,8,8
9,9,9,9,9,9,9,9,9,304.509499,303.882124,...,303.186503,9,9,9,9,9,9,9,9,9


In [22]:
agri = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm2/landcover/agriculture_land_wide.csv")

In [23]:
agri.rename(columns = {"cityCode" : "id"}, inplace=True)

In [24]:
agri = agri[agri.columns[agri.columns.str.contains("agr|id")]]

In [25]:
agri.columns

Index(['id', 'agr_2001', 'agr_2002', 'agr_2003', 'agr_2004', 'agr_2005',
       'agr_2006', 'agr_2007', 'agr_2008', 'agr_2009', 'agr_2010', 'agr_2011',
       'agr_2012', 'agr_2013', 'agr_2014', 'agr_2015', 'agr_2016', 'agr_2017',
       'agr_2018', 'agr_2019', 'agr_2020', 'agr_2021', 'agr_2022'],
      dtype='object')

In [26]:
urba = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm2/landcover/urban_land_wide.csv")

In [27]:
urba.rename(columns = {"cityCode" : "id"}, inplace=True)

In [28]:
urba = urba[urba.columns[urba.columns.str.contains("urb|id")]]

In [29]:
ntl = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm2/ntl/ntl_sum_wide_00-22.csv")

In [30]:
ntl = ntl[ntl.columns[ntl.columns.str.contains("ntl|id")]]

In [31]:
datasets = [prep, temp, agri, urba, ntl]

In [32]:
municipality_data = pd.merge(sez, pop_sum[pop_sum.columns[pop_sum.columns.str.contains("pop|id")]], on="id", how="outer")

In [33]:
for dataset in datasets: 
    municipality_data = pd.merge(municipality_data, dataset, on="id", how= "outer")

In [34]:
filter_year = ["2017", "2018", "2018", "2019", "2020", "2021", "2022", "2023","1999"]

pattern = "|".join(filter_year)
municipality_data = municipality_data.drop(columns = municipality_data.columns[municipality_data.columns.str.contains(pattern)])

In [35]:
#municipality_data.to_csv("municipality_data.csv", index=False)

In [36]:
var_names = ["inv", "ob_f_","tec_f_","adm_f_","ob_m_","tec_m_","adm_m_",'ent', 'ele', "com", 
"inf", "wat", "tss", "buiilt", "ocu", "sal_op", "sal_tec", "pop", "prep", "temp", "agr_", "urb_", "ntl_"]

In [37]:
pattern2 = "|".join(var_names)

In [38]:
long_df = pd.wide_to_long(municipality_data, stubnames=var_names, i='id', j='year', sep='')


In [39]:
long_df = long_df[long_df.columns[long_df.columns.str.contains("shape|id|year")|long_df.columns.str.contains(pattern2)]]

In [40]:
long_df =  long_df.drop(columns = "id.1")
long_df =  long_df.drop(columns = "sez_id")

In [41]:
long_df.dtypes

shapeName     object
year         float64
inv          float64
ob_f_        float64
tec_f_       float64
adm_f_       float64
ob_m_        float64
tec_m_       float64
adm_m_       float64
ent          float64
ele          float64
com          float64
inf          float64
wat          float64
tss          float64
buiilt       float64
ocu          float64
sal_op       float64
sal_tec      float64
pop          float64
prep         float64
temp         float64
agr_         float64
urb_         float64
ntl_         float64
dtype: object

In [42]:
long_df.to_csv("long_df.csv")